In [ ]:
import os
import pandas as pd

pd.options.display.max_columns = 100

os.listdir('inputs')

In [ ]:
train = pd.read_csv('inputs/train.csv')
test = pd.read_csv('inputs/test.csv')
sub = pd.read_csv('inputs/sampleSubmission.csv')

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
train['Category'].value_counts()

In [ ]:
all_df = pd.concat([train, test])

In [ ]:
all_df['year'] = all_df['Dates'].astype('datetime64').dt.year
all_df['month'] = all_df['Dates'].astype('datetime64').dt.month
all_df['day'] = all_df['Dates'].astype('datetime64').dt.day

all_df['hour'] = all_df['Dates'].astype('datetime64').dt.hour
all_df['minute'] = all_df['Dates'].astype('datetime64').dt.minute
all_df['all_minute'] = all_df['hour'] * 60 + all_df['minute']



In [ ]:
left_top_X = all_df['X'].min()
right_top_X = all_df['X'].max()
left_bottom_X = all_df['X'].min()
right_bottom_X = all_df['X'].max()

left_top_Y = all_df['Y'].max()
right_top_Y = all_df['Y'].max()
left_bottom_Y = all_df['Y'].min()
right_bottom_Y = all_df['Y'].min()

In [ ]:
all_df['distance_1'] = (((all_df['X'] - right_top_X) ** 2 )+ ((all_df['Y'] - right_top_Y) ** 2))**0.5 # right_top 1사분면
all_df['distance_2'] = (((all_df['X'] - left_top_X) ** 2 )+ ((all_df['Y'] - left_top_Y) ** 2))**0.5 # left_top 2사분면
all_df['distance_3'] = (((all_df['X'] - left_bottom_X) ** 2 )+ ((all_df['Y'] - left_bottom_Y) ** 2))**0.5 # left_bottom
all_df['distance_4'] = (((all_df['X'] - right_bottom_X) ** 2 )+ ((all_df['Y'] - right_bottom_Y) ** 2))**0.5 # right_bottom

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.distplot(all_df.all_minute)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.distplot(all_df[len(train):].all_minute)

In [ ]:
all_df.head()

In [ ]:
all_df.tail()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit_transform(list(all_df['Category']))

In [ ]:
le_classes = le.classes_

In [ ]:
all_df.dtypes

In [ ]:
for i in all_df.columns[all_df.dtypes == 'object']:
    if i == 'Category' or i == 'Dates' or i == 'Descript' or i == 'Resolution':
        continue # object 형 컬럼 중 라벨인코딩에 필요없는건 넘겨줄 것.(나중에 모델에 들어가기 전에 빼주면 됨.(drop(['date', 'id'], axis=1)). 원본을 남길 필요가 있는 경우 이처럼 continue 시켜줌)
    all_df[i] = le.fit_transform(list(all_df[i])) # list로 감싸주면 na값이 문자 'NaN'로 처리되서 오류 안 뜸.

In [ ]:
all_df.dtypes

In [ ]:
all_df = all_df.drop(['Dates', 'Category', 'Descript', 'Resolution', 'Id', 'Address'], axis = 1)

train2 = all_df[:len(train)]
test2 = all_df[len(train):]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train2, train['Category'], test_size = 0.2, random_state = 33, stratify=train['Category'])

In [ ]:
train2

In [ ]:
test2

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier()
# rf.fit(train2, train['Category'])
# result = rf.predict_proba(test2)

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(n_estimators = 100, learning_rate = 0.01, num_leaves = 64)
lgb.fit(x_train, y_train, eval_set = (x_valid, y_valid), verbose_eval=20)
result1 = lgb.predict_proba(test2)

In [ ]:
from catboost import CatBoostClassifier

cb = CatBoostClassifier(n_estimators = 1000, learning_rate = 0.05)
cb.fit(x_train, y_train, eval_set = (x_valid, y_valid), verbose_eval=20)
result2 = cb.predict_proba(test2)

In [ ]:
sub.iloc[:, 1:] = 0.5*result1 + 0.5*result2

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submissions/sub.csv', index=False)

https://www.kaggle.com/c/sf-crime/submit

- 베이스라인(아무 처리 안함, result 값만 도출되게끔 했을때): 3.87207
- validation 나눠서 2.54469 lgb val score는 2.50426
- learning_rate: 0.05로 낮췄더니 2.40392 lgb val score는 2.50078 이번엔 오히려 lb 점수가 더 좋음. 모델 예측 점수와 너무 다름
- catboost로 했더니 2.32982 val은 2.3231609 거의 비슷함
- lgb랑 catboost랑 앙상블 했더니 점수 더 떨어짐 - 2.37649
